In [3]:
#knock60
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(r"C:\Users\Ayuto\Downloads\GoogleNews-vectors-negative300.bin.gz",binary=True)
print(model["United_States"])

[-3.61328125e-02 -4.83398438e-02  2.35351562e-01  1.74804688e-01
 -1.46484375e-01 -7.42187500e-02 -1.01562500e-01 -7.71484375e-02
  1.09375000e-01 -5.71289062e-02 -1.48437500e-01 -6.00585938e-02
  1.74804688e-01 -7.71484375e-02  2.58789062e-02 -7.66601562e-02
 -3.80859375e-02  1.35742188e-01  3.75976562e-02 -4.19921875e-02
 -3.56445312e-02  5.34667969e-02  3.68118286e-04 -1.66992188e-01
 -1.17187500e-01  1.41601562e-01 -1.69921875e-01 -6.49414062e-02
 -1.66992188e-01  1.00585938e-01  1.15722656e-01 -2.18750000e-01
 -9.86328125e-02 -2.56347656e-02  1.23046875e-01 -3.54003906e-02
 -1.58203125e-01 -1.60156250e-01  2.94189453e-02  8.15429688e-02
  6.88476562e-02  1.87500000e-01  6.49414062e-02  1.15234375e-01
 -2.27050781e-02  3.32031250e-01 -3.27148438e-02  1.77734375e-01
 -2.08007812e-01  4.54101562e-02 -1.23901367e-02  1.19628906e-01
  7.44628906e-03 -9.03320312e-03  1.14257812e-01  1.69921875e-01
 -2.38281250e-01 -2.79541016e-02 -1.21093750e-01  2.47802734e-02
  7.71484375e-02 -2.81982

In [4]:
#knock61
from gensim.models import KeyedVectors
print(model.similarity('United_States', 'U.S.'))

0.73107743


In [5]:
#knock62
print(*model.most_similar("United_States", topn=10), sep="\n")

('Unites_States', 0.7877248525619507)
('Untied_States', 0.7541370987892151)
('United_Sates', 0.7400724291801453)
('U.S.', 0.7310774326324463)
('theUnited_States', 0.6404393911361694)
('America', 0.6178410053253174)
('UnitedStates', 0.6167312264442444)
('Europe', 0.6132988929748535)
('countries', 0.6044804453849792)
('Canada', 0.601906955242157)


In [4]:
#knock63
spain = model["Spain"]
madrid = model["Madrid"]
athens = model["Athens"]
print(*model.most_similar(spain - madrid + athens, topn=10), sep="\n")

('Athens', 0.7528455853462219)
('Greece', 0.6685472130775452)
('Aristeidis_Grigoriadis', 0.5495778322219849)
('Ioannis_Drymonakos', 0.5361457467079163)
('Greeks', 0.5351786017417908)
('Ioannis_Christou', 0.5330225825309753)
('Hrysopiyi_Devetzi', 0.5088489055633545)
('Iraklion', 0.5059264302253723)
('Greek', 0.5040615797042847)
('Athens_Greece', 0.5034108757972717)


In [6]:
#knock64
with open('questions-words.txt', 'r') as f1, open('questions-words-add.txt', 'w', encoding="utf-8") as f2:
  for line in f1:  
    line = line.split()
    if line[0] == ':':
      category = line[1]
    else:
      word, cos = model.most_similar(positive=[line[1], line[2]], negative=[line[0]], topn=1)[0]
      f2.write(' '.join([category] + line + [word, str(cos) + '\n']))

In [7]:
#knock65
with open('./questions-words-add.txt', 'r') as f:
        sem_cnt = 0
        sem_cor = 0
        syn_cnt = 0
        syn_cor = 0
        for line in f:
            line = line.split()
            if not line[0].startswith('gram'):
                sem_cnt += 1
            if line[4] == line[5]:
                sem_cor += 1
            else:
                syn_cnt += 1
            if line[4] == line[5]:
                syn_cor += 1

print(f'意味的アナロジー正解率: {sem_cor/sem_cnt:.3f}')
print(f'文法的アナロジー正解率: {syn_cor/syn_cnt:.3f}')

意味的アナロジー正解率: 1.622
文法的アナロジー正解率: 2.786


In [6]:
#knock66
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from tqdm import tqdm
tqdm.pandas()


def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


def calc_cos_sim(row):
    w1v = model[row['Word 1']]
    w2v = model[row['Word 2']]
    return cos_sim(w1v, w2v)

combined_df = pd.read_csv('combined.csv')

combined_df['cos_sim'] = combined_df.progress_apply(calc_cos_sim, axis=1)
spearman_corr = combined_df[['Human (mean)', 'cos_sim']].corr(method='spearman')
print(f'spearman corr: {spearman_corr}')

100%|██████████| 353/353 [00:00<00:00, 75413.30it/s]

spearman corr:               Human (mean)   cos_sim
Human (mean)      1.000000  0.700017
cos_sim           0.700017  1.000000


In [8]:
#knock67データの準備
# https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who?resource=download
life_df = pd.read_csv('Life Expectancy Data.csv')
countries_df = life_df['Country'].unique()
countries = countries_df.tolist()
countries = [country.replace(" ", "_") for country in countries]

In [23]:
#knock67
from sklearn.cluster import KMeans

countries_vec = []
for country in countries:
    try:
        countries_vec.append(model[country])
    except:
        #print("not exist:", country)
        continue
#print(countries_vec)


not exist: Antigua_and_Barbuda
not exist: Bolivia_(Plurinational_State_of)
not exist: Bosnia_and_Herzegovina
not exist: Côte_d'Ivoire
not exist: Central_African_Republic
not exist: Democratic_People's_Republic_of_Korea
not exist: Democratic_Republic_of_the_Congo
not exist: Guinea-Bissau
not exist: Iran_(Islamic_Republic_of)
not exist: Lao_People's_Democratic_Republic
not exist: Micronesia_(Federated_States_of)
not exist: Papua_New_Guinea
not exist: Republic_of_Korea
not exist: Republic_of_Moldova
not exist: Russian_Federation
not exist: Saint_Kitts_and_Nevis
not exist: Saint_Vincent_and_the_Grenadines
not exist: Sao_Tome_and_Principe
not exist: South_Sudan
not exist: Syrian_Arab_Republic
not exist: The_former_Yugoslav_republic_of_Macedonia
not exist: Timor-Leste
not exist: Trinidad_and_Tobago
not exist: United_Kingdom_of_Great_Britain_and_Northern_Ireland
not exist: United_Republic_of_Tanzania
not exist: United_States_of_America
not exist: Venezuela_(Bolivarian_Republic_of)


In [22]:
#knock67 k-meansクラスタリング
kmeans = KMeans(n_clusters=5, random_state=172957)
kmeans.fit(countries_vec)
for i in range(5):
    cluster = np.where(kmeans.labels_ == i)[0]
    print('cluster', i)
    print(', '.join([countries[k] for k in cluster]))

cluster 0
Algeria, Angola, Belize, Bhutan, Brazil, Brunei_Darussalam, Burundi, Cabo_Verde, Central_African_Republic, Chad, Croatia, Democratic_People's_Republic_of_Korea, Denmark, Djibouti, Dominican_Republic, Equatorial_Guinea, Eritrea, Fiji, Gambia, Iran_(Islamic_Republic_of), Japan, Jordan, Kazakhstan, Kuwait, Kyrgyzstan, Lebanon, Libya, Maldives, Mali, Marshall_Islands, Mongolia, Montenegro, Oman, Paraguay, Poland, Romania, Russian_Federation, Saint_Lucia, Samoa, Serbia, Sierra_Leone, Solomon_Islands, Suriname, Swaziland, Sweden
cluster 1
Armenia, Azerbaijan, Bangladesh, Belgium, Bulgaria, Chile, Cuba, Ecuador, France, Georgia, Hungary, Iraq, Latvia, Liberia, Lithuania, Mauritania, Micronesia_(Federated_States_of), Morocco, Nauru, Pakistan, Palau, Philippines, Republic_of_Moldova, Seychelles, Slovenia, Sri_Lanka
cluster 2
Albania, Argentina, Australia, Barbados, Belarus, Botswana, Côte_d'Ivoire, Colombia, Congo, Cook_Islands, Costa_Rica, Dominica, Egypt, El_Salvador, Estonia, Ethio

c:\Users\Ayuto\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [27]:
#knock68
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

plt.figure(figsize=(15, 5))
Z = linkage(countries_vec, method='ward')
dendrogram(Z, labels=countries)
plt.show()

In [ ]:
#knock69
import bhtsne

embedded = bhtsne.tsne(np.array(countries_vec).astype(np.float64), dimensions=2, rand_seed=123)
plt.figure(figsize=(10, 10))
plt.scatter(np.array(embedded).T[0], np.array(embedded).T[1])
for (x, y), name in zip(embedded, countries):
    plt.annotate(name, (x, y))
plt.show()